In [4]:
import json
import numpy as np
from functools import partial
from collections import Counter

In [5]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_original.json") as f:
    data = json.load(f)

In [6]:
len(data)

94798

In [7]:
data[0].keys()

dict_keys(['object_indices', 'scan', 'object_name', 'path_id', 'trajectories', 'end_regions', 'paths', 'instructions', 'start_region_name', 'heading', 'end_region_name', 'start_region', 'distances'])

In [8]:
# most common of all
instr_set = set()
instr_list = []
for d in data:
    instr_set.add(d['instructions'][0])
    instr_list.append(d['instructions'][0])

len(instr_set)

1444

In [9]:
Counter(instr_list).most_common(10)

[('find a sink in the kitchen .', 650),
 ('find a refrigerator in the kitchen .', 626),
 ('find a couch in the living room .', 614),
 ('find a mirror in one of the bedrooms .', 589),
 ('find a picture in the living room .', 586),
 ('find a table in the living room .', 579),
 ('find a chair in one of the bedrooms .', 566),
 ('find a nightstand in one of the bedrooms .', 560),
 ('find a kitchen counter in the kitchen .', 533),
 ('find a lamp in one of the bedrooms .', 513)]

In [10]:
# Figure 9 number of actions 5-9
def filter_action_lens(low, high, d):
    mean_path_len = np.mean([len(path) for path in d['paths']])
    if low <= mean_path_len and high >= mean_path_len:
        return True
    else:
        return False

filter_5_9 = partial(filter_action_lens, 5, 9)

# -------------------------------------------------------
# Figure 5 start rooms
def filter_start_room(start_rm_list, d):
    if d['start_region_name'] in start_rm_list:
        return True
    else:
        return False
    
filter_start_hallway = partial(filter_start_room, ['hallway'])

# -------------------------------------------------------
# Figure 5 goal rooms
def filter_goal_room(goal_rm_list, d):
    if d['end_region_name'] in goal_rm_list:
        return True
    else:
        return False
    
filter_end_bedroom = partial(filter_goal_room, ['bedroom'])
filter_end_bathroom = partial(filter_goal_room, ['bathroom'])
filter_end_common = partial(filter_goal_room, ['bedroom', 'bathroom'])
# -------------------------------------------------------
# Figure 4 objects
def filter_object(object_list, d):
    if d['object_name'] in object_list:
        return True
    else:
        return False
    
filter_picture = partial(filter_object, ['picture'])
filter_mirror = partial(filter_object, ['mirror'])
filter_pillow = partial(filter_object, ['pillow'])
filter_picture_mirror_pillow = partial(filter_object, ['mirror', 'picture', 'pillow'])
# -------------------------------------------------------
# Language instruction
def filter_instruction(instr_str_list, d):
    for instr_str in instr_str_list:
        if instr_str in d['instructions'][0]:
            return True
    return False

filter_instruction_most_common =  partial(filter_instruction, ['find a sink in the kitchen '])
# -------------------------------------------------------
def filter_simple_case(d):
    if filter_pillow(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_bedroom(d):
        return True
    else:
        return False
    
def filter_simple_case2(d):
    if filter_mirror(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_bathroom(d):
        return True
    else:
        return False

def filter_common1(d):
    if filter_picture_mirror_pillow(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_bathroom(d):
        return True
    else:
        return False
    
def filter_common2(d):
    if filter_picture_mirror_pillow(d) and filter_5_9(d) and filter_start_hallway(d) and filter_end_common(d):
        return True
    else:
        return False
    
def filter_common3(d):
    if filter_instruction_most_common(d) and filter_5_9(d):
        return True
    else:
        return False

In [11]:
# 5-9, start hallway, end bedroom, pillow
len(list(filter(filter_simple_case, data)))

26

In [12]:
# 5-9, start hallway, end bathroom, mirror
len(list(filter(filter_simple_case2, data)))

34

In [13]:
# 5-9, start hallway, end bathroom, mirror/picture/pillow
len(list(filter(filter_common1, data)))

70

In [14]:
# 5-9, start hallway, end bathroom, mirror/picture/pillow
len(list(filter(filter_common2, data)))

173

In [15]:
# 5-9, 'find a sink in the kitchen '
len(list(filter(filter_common3, data)))

372

In [16]:
# any length, 'find a sink in the kitchen '
len(list(filter(filter_instruction_most_common, data)))

650

# extract by indices

In [17]:
# 5-9, 'find a sink in the kitchen '
data_common3 = [data[idx] for idx in range(len(data)) if filter_common3(data[idx])]
data_common3_idx = [idx for idx in range(len(data)) if filter_common3(data[idx])]

In [76]:
scans_common3 = [d['scan'] for d in data_common3]
Counter(scans_common3).most_common(10)

[('ac26ZMwG7aT', 38),
 ('8WUmhLawc2A', 37),
 ('cV4RVeZvu5T', 26),
 ('JmbYfDe2QKZ', 26),
 ('mJXqzFtmKg4', 23),
 ('VzqfbhrpDEA', 23),
 ('EDJbREhghzL', 22),
 ('82sE5b5pLXE', 22),
 ('gTV8FGcVJC9', 21),
 ('e9zR4mvMWw7', 21)]

In [78]:
examine_d = [d for d in data_common3 if d['scan'] == 'ac26ZMwG7aT']

In [81]:
print (examine_d[0]['scan'])
print (examine_d[37]['scan'])

ac26ZMwG7aT
ac26ZMwG7aT


In [114]:
# [d['path_id'] for d in examine_d]

In [112]:
examine_d[0]

{'distances': [17.83902011259488],
 'end_region_name': 'kitchen',
 'end_regions': [7],
 'heading': 5.235987755982989,
 'instructions': ['find a sink in the kitchen .'],
 'object_indices': [[348, 355]],
 'object_name': 'sink',
 'path_id': 156477,
 'paths': [['13faa4c86406405ba802f5bdcba744ee',
   '11fae969386e4e4fa6ed73ac9f20cd1d',
   '735cc8eae07743a1aa5d41ba878b0eb4',
   '057625814959401f8e969f5bfbf6cca0',
   '91b29545634c4754bfe914564d6c950f',
   'effea130a5984e53bd2bef2bbe4ba4c1',
   '57b84179a9a24ff49697d9349a21bc60']],
 'scan': 'ac26ZMwG7aT',
 'start_region': 25,
 'start_region_name': 'bedroom',
 'trajectories': [[[0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, -1, 0],
   [0, -1, 0],
   [1, 0, 0],
   [0, -1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, 1, 0],
   [0, 1, 0],
   [2, 0, 0],
   [0, 0, 0]]]}

In [88]:
[d['paths'][0][-1] for d in examine_d]

['57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 '57b84179a9a24ff49697d9349a21bc60',
 

In [92]:
[len(d['paths'][0]) for d in examine_d]

[7,
 9,
 9,
 5,
 7,
 9,
 5,
 9,
 7,
 8,
 5,
 7,
 6,
 5,
 6,
 6,
 7,
 6,
 6,
 5,
 8,
 9,
 5,
 7,
 6,
 6,
 8,
 6,
 6,
 9,
 9,
 7,
 6,
 9,
 8,
 5,
 9,
 5]

## Single example dataset

In [99]:
type(data)

list

In [106]:
# scan 'ac26ZMwG7aT', length 5, 'find a sink in the kitchen '
examine_d[-1]

{'distances': [11.124175652960771],
 'end_region_name': 'kitchen',
 'end_regions': [7],
 'heading': 5.235987755982989,
 'instructions': ['find a sink in the kitchen .'],
 'object_indices': [[348, 355]],
 'object_name': 'sink',
 'path_id': 156482,
 'paths': [['828b8f02ffd145ac8b351cbb70682dc2',
   '7fb0cc6c0754461ca29084c1d793b600',
   '96b25d845ff447cc989dfd87c88ff8b8',
   'fbc5ba2686d24a2d86cc1ac0575325ae',
   '57b84179a9a24ff49697d9349a21bc60']],
 'scan': 'ac26ZMwG7aT',
 'start_region': 0,
 'start_region_name': 'other room',
 'trajectories': [[[0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [0, 1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, 1, 0],
   [1, 0, 0],
   [1, 0, 0],
   [0, 0, 0]]]}

In [107]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_small_single_example.json", "w") as f:
    json.dump([examine_d[-1]]*100, f)

## Single house, short, same goal example dataset

In [94]:
# scan 'ac26ZMwG7aT', 5-9, 'find a sink in the kitchen '
len(examine_d)

38

In [104]:
type(examine_d)

list

In [105]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_small_single_house_single_goal_short.json", "w") as f:
    json.dump(examine_d, f)

## Single house, any length, same goal example dataset

In [95]:
# any length, 'find a sink in the kitchen '
data_common4 = [data[idx] for idx in range(len(data)) if filter_instruction_most_common(data[idx])]
data_common4_idx = [idx for idx in range(len(data)) if filter_instruction_most_common(data[idx])]

In [97]:
# scan 'ac26ZMwG7aT', any length, 'find a sink in the kitchen '
examine_e = [d for d in data_common4 if d['scan'] == 'ac26ZMwG7aT']
len(examine_e)

59

## Three houses, short, same goal example dataset

In [95]:
# any length, 'find a sink in the kitchen '
data_common5 = [data[idx] for idx in range(len(data)) if filter_instruction_most_common(data[idx])]
data_common5_idx = [idx for idx in range(len(data)) if filter_instruction_most_common(data[idx])]

In [19]:
#  ('ac26ZMwG7aT', 38),
#  ('8WUmhLawc2A', 37),
#  ('cV4RVeZvu5T', 26)
#  3 houses, 5-9, 'find a sink in the kitchen '
examine_f = [d for d in data_common3 if d['scan'] in ['ac26ZMwG7aT', '8WUmhLawc2A', 'cV4RVeZvu5T']]
len(examine_f)

101

In [29]:
# short path length by vertex ID
len(examine_f[91]['paths'][0])

9

In [23]:
# long trajectory length by tota=l number of gold actions
len(examine_f[91]['trajectories'][0])

20

In [26]:
len(examine_f[90]['trajectories'][0])

13

In [24]:
len(examine_f[91]['trajectories'][1])

IndexError: list index out of range

In [118]:
with open("/home/hoyeung/blob_matterport3d/asknav/asknav_train_small_three_houses_single_goal_short.json", "w") as f:
    json.dump(examine_f, f)

## Multiple houses, any length, same goal example dataset

In [98]:
# any scans, any length, 'find a sink in the kitchen '
len(data_common4)

650

In [119]:
for _ in range(11): print("ok")

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


In [120]:
l = [[None]] * 10
l

[[None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None]]

In [121]:
l[0].append(1)
l

[[None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1],
 [None, 1]]

In [122]:
l = [[None]*3] * 10
l

[[None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [123]:
l[0].append(1)
l

[[None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1],
 [None, None, None, 1]]

SyntaxError: invalid syntax (<ipython-input-124-cbdf10b85b89>, line 1)

In [125]:
l = [None] * 10

In [126]:
l

[None, None, None, None, None, None, None, None, None, None]

In [127]:
l[0] = 1
l

[1, None, None, None, None, None, None, None, None, None]

In [129]:
l = [[].copy()] * 10

In [130]:
l

[[], [], [], [], [], [], [], [], [], []]

In [131]:
l[0].append(1)
l

[[1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]

In [133]:
l = [[] for _ in range(10)]
l

[[], [], [], [], [], [], [], [], [], []]

In [134]:
l[0] = 1

In [135]:
l

[1, [], [], [], [], [], [], [], [], []]

In [139]:
d = [[None for s in range(3)] for _ in range(10)]
d

[[None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [140]:
d[0][1] = [1,1,1]
d

[[None, [1, 1, 1], None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None],
 [None, None, None]]

In [ ]:
np.ones(len(obs), len(viewix_next_vertex_map[0])) * 1e9